## MNIST with CNN

In [1]:
import tensorflow as tf

/home/hdsingh/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

## Helper Functions

In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [17]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [6]:
def conv2d(x,W):
    # x -> [batch,H,W,Channels]
    # W -> [filter H, filter W, Channels In, Channels Out]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [11]:
def max_pool_2by2(x):
    # x - >[batch,h,w,c]
    return tf.nn.max_pool(x,ksize= [1,2,2,1],
                          strides= [1,2,2,1],
                          padding= 'SAME',)

In [19]:
def conv_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [23]:
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+b 

In [14]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape = [None,10])

In [15]:
#Layers
x_image = tf.reshape(x,[-1,28,28,1])

In [24]:
convo_1 = conv_layer(x_image,shape = [5,5,1,32])
# [patch_h,patch_w,channels_in,channels_out]
convo_1_pooling = max_pool_2by2(convo_1)

In [25]:
convo_2 = conv_layer(convo_1_pooling,shape = [5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [26]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

### Dropout

In [27]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,
                                 keep_prob = hold_prob)

In [28]:
y_pred = normal_full_layer(full_one_dropout,10)

### Loss Function

In [30]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [31]:
optimizer = tf.train.AdamOptimizer(0.001)
train = optimizer.minimize(cross_entropy)

In [32]:
init = tf.global_variables_initializer()

### Session

In [35]:
steps = 5000

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict  = {x: batch_x, y_true:batch_y,hold_prob: 0.5 })
        
        if i%100 ==0:
            print('ON STEP: {}'.format(i))
            print('ACCURACY: ')
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict = {x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

ON STEP: 0
ACCURACY: 
0.1387


ON STEP: 100
ACCURACY: 
0.9382


ON STEP: 200
ACCURACY: 
0.9647


ON STEP: 300
ACCURACY: 
0.9709


ON STEP: 400
ACCURACY: 
0.9751


ON STEP: 500
ACCURACY: 
0.9765


ON STEP: 600
ACCURACY: 
0.979


ON STEP: 700
ACCURACY: 
0.9794


ON STEP: 800
ACCURACY: 
0.982


ON STEP: 900
ACCURACY: 
0.9817


ON STEP: 1000
ACCURACY: 
0.9843


ON STEP: 1100
ACCURACY: 
0.9846


ON STEP: 1200
ACCURACY: 
0.9859


ON STEP: 1300
ACCURACY: 
0.9847


ON STEP: 1400
ACCURACY: 
0.9841


ON STEP: 1500
ACCURACY: 
0.987


ON STEP: 1600
ACCURACY: 
0.9867


ON STEP: 1700
ACCURACY: 
0.9855


ON STEP: 1800
ACCURACY: 
0.9875


ON STEP: 1900
ACCURACY: 
0.9877


ON STEP: 2000
ACCURACY: 
0.9884


ON STEP: 2100
ACCURACY: 
0.9861


ON STEP: 2200
ACCURACY: 
0.9884


ON STEP: 2300
ACCURACY: 
0.9889


ON STEP: 2400
ACCURACY: 
0.9884


ON STEP: 2500
ACCURACY: 
0.9854


ON STEP: 2600
ACCURACY: 
0.9885


ON STEP: 2700
ACCURACY: 
0.9892


ON STEP: 2800
ACCURACY: 
0.9884


ON STEP: 2900
ACCURACY: 
0.98